# Búsqueda vectorial


In [ ]:
import json

with open('embeddings/peliculas_text-embedding-3-small-1536.json') as f:
    movies = json.load(f)

MODEL_DIMENSIONS = 1536 # Must match the dimension of the embeddings

## Configurar el OpenAI client con GitHub Models

Ejecuta la celda de abajo si estás utilizando OpenAI con GitHub Models. Si estás ejecutando esto en GitHub Codespaces, la variable de entorno GITHUB_TOKEN ya estará configurada para ti. Si estás ejecutando esto localmente, asegúrate de configurar la variable de entorno GITHUB_TOKEN con un Token de Acceso Personal (PAT) de GitHub.

In [ ]:
import os
import openai

openai_client = openai.OpenAI(
    base_url="https://models.github.ai/inference",
    api_key=os.environ["GITHUB_TOKEN"]
)
MODEL_NAME = "openai/text-embedding-3-small"

## Configurar el OpenAI client con modelos de Azure OpenAI

Ejecuta la celda de abajo si estás utilizando Azure OpenAI keyless auth y ya has configurado un archivo .env con las variables requeridas.

In [ ]:
import os

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

azure_credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = OpenAI(
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT") + "/openai/v1/",
    api_key=token_provider,
)
MODEL_NAME = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

## Defina la función para generar un embedding

In [ ]:
def get_embedding(text):
    embeddings_response = openai_client.embeddings.create(
        model=MODEL_NAME,
        dimensions=MODEL_DIMENSIONS,
        input=text,
    )
    return embeddings_response.data[0].embedding

## Busqueda de embeddings vectorial

In [ ]:
import pandas as pd

def cosine_similarity(v1, v2):
    dot_product = sum([a * b for a, b in zip(v1, v2)])
    magnitude = (sum([a**2 for a in v1]) * sum([a**2 for a in v2])) ** 0.5
    return dot_product / magnitude

def exhaustive_search(query_vector, vectors):
    similarities = []
    for title, vector in vectors.items():
        similarity = cosine_similarity(query_vector, vector)
        similarities.append((title, similarity))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities

new_vector = get_embedding("una pelicula apta para ninos acerca de animales")
similarities = exhaustive_search(new_vector, movies)
most_similar = similarities[0:10]
similar_movies = [(movie, round(similarity, 3)) for movie, similarity in most_similar]

pd.DataFrame(similar_movies, columns=['pelicula', 'similitud'])

## Búsqueda ANN: HNSW


In [ ]:
import hnswlib

# Declarando índice
p = hnswlib.Index(space='cosine', dim=1536)

# Inicializando índice - el número máximo de elementos debe conocerse de antemano
p.init_index(max_elements=len(movies), ef_construction=200, M=16)

# Inserción de elementos (puede ser llamada varias veces):
vectors = list(movies.values())
ids = list([i for i in range(len(vectors))])
p.add_items(vectors, ids)

# Controlando la recuperación mediante el ajuste de ef:
p.set_ef(50) # ef siempre debe ser > k

### Los parámetros del índice se exponen como propiedades de clase:
print(f"Parámetros pasados al constructor:  space={p.space}, dim={p.dim}") 
print(f"Construcción del índice: M={p.M}, ef_construction={p.ef_construction}")
print(f"El tamaño del índice es {p.element_count} y la capacidad del índice es {p.max_elements}")
print(f"Parámetro de equilibrio entre velocidad/calidad de búsqueda: ef={p.ef}")

In [ ]:
# Buscar en el índice HNSW
new_vector = get_embedding("una pelicula apta para ninos acerca de gatos")

labels, distances = p.knn_query(new_vector, k=10)

# asociar etiquetas con títulos de películas y mostrarlos
similar_movies = [(list(movies.keys())[label], round(1 - distance, 3)) for label, distance in zip(labels[0], distances[0])]
pd.DataFrame(similar_movies, columns=['película', 'similitud'])